### Importing the libraries

In [1]:
import numpy as np
import tensorflow as tf
import re
import time

### Importing the dataset

In [2]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')


In [3]:
lines

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [4]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

#### Creating the dictionary that maps each line and its id

In [5]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [6]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

##### Create a list of all the conversation

In [7]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))
    
conversations_ids
    
    

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

#### Get separately the questions and the answers

In [8]:
questions = []
answers = []

for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])
#     x = conversation[::2]
#     y = conversation[1::2]



In [9]:
questions

['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
 "Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "You're asking me out.  That's so cute. What's your name again?",
 "No, no, it's my fault -- we didn't have a proper introduction ---",
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Why?',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 'Gosh, if only we could find Kat a boyfriend...',
 "C'esc ma tete. This is my head",
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have n

In [10]:
answers

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

#### Cleaning the texts

In [11]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text) # remove apostrophe
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

#### Cleaning the questions and answers



In [12]:
clean_questions = []
clean_answers = []

for question in questions:
    clean_questions.append(clean_text(question))
    

for answer in answers:
    clean_answers.append(clean_text(answer))
    
    

In [13]:
clean_questions

['can we make this quick  roxanne korrine and andrew barrett are having an incredibly horrendous public break up on the quad  again',
 'well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 'you are asking me out  that is so cute what is your name again',
 "no no it's my fault  we didn't have a proper introduction ",
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does',
 'why',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'gosh if only we could find kat a boyfriend',
 "c'esc ma tete this is my head",
 'right  see  you are ready for the quiz',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out

In [ ]:
clean_answers

### Remove non frequent words

#### Creating a dictionary that maps each word to its number of occurences

In [ ]:
word2count={} #it maps each word to how many time it appears

for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [ ]:
word2count

In [ ]:
# qq = ' Hi How are you? Hi Hi '
# pq = {}
# for word in qq.split():
# #     print(word)
#     if word not in pq:
#         pq[word] = 1
#     else:
#         pq[word] += 1

In [ ]:
word2count.get('if')

#### Creating two dictionaries that map the questions words and the answers words to a unique integer

`We perform tokenization here`

* remove 5% least frequent words

In [ ]:
threshold_questions = 20 
questionswords2int = {}
word_number = 0

"""Include 95% of most frequent word of question and at the same time 
associate them to a uniue integer."""
for word,count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number # include the word in dictionary
        word_number += 1
        
# questionsword2int


threshold_answers = 20 
answerswords2int = {}
word_number = 0

"""Include 95% of most frequent word of answers and at the same time 
associate them to a uniue integer."""
for word,count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number # include the word in dictionary
        word_number += 1
        
# answersword2int

In [ ]:
print(len(answerswords2int))
answerswords2int['avoiding']

In [ ]:
print(len(questionswords2int))
# print(questionswords2int['Roxanne'])
questionswords2int

In [ ]:
print(len(answerswords2int))
answerswords2int

### Adding the list of tokens to these two dictionaries, that is sos and eos

In [23]:
tokens = ['<PAD>','<EOS>','<OUT>','<SOS>'] # OUT means all the words that were filtered out by all the previous dictionries

for token in tokens:
    questionswords2int[token] = len(questionswords2int) + 1
    
for token in tokens:
    answerswords2int[token] = len(answerswords2int) + 1

In [24]:
len(questionswords2int)

8825

In [25]:
len(answerswords2int)

8825

### Creating the inverse dictionary of the answerswords2int dictionary

In [26]:
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}
answersints2word

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'quick',
 5: 'and',
 6: 'andrew',
 7: 'are',
 8: 'having',
 9: 'an',
 10: 'incredibly',
 11: 'public',
 12: 'break',
 13: 'up',
 14: 'on',
 15: 'the',
 16: 'again',
 17: 'well',
 18: 'i',
 19: 'thought',
 20: 'would',
 21: 'start',
 22: 'with',
 23: 'if',
 24: 'that',
 25: 'is',
 26: 'okay',
 27: 'you',
 28: 'not',
 29: 'part',
 30: 'please',
 31: 'asking',
 32: 'me',
 33: 'out',
 34: 'so',
 35: 'cute',
 36: 'what',
 37: 'your',
 38: 'name',
 39: 'no',
 40: "it's",
 41: 'my',
 42: 'fault',
 43: "didn't",
 44: 'have',
 45: 'a',
 46: 'proper',
 47: 'cameron',
 48: 'thing',
 49: 'am',
 50: 'at',
 51: 'mercy',
 52: 'of',
 53: 'particularly',
 54: 'breed',
 55: 'loser',
 56: 'sister',
 57: 'cannot',
 58: 'date',
 59: 'until',
 60: 'she',
 61: 'does',
 62: 'why',
 63: 'mystery',
 64: 'used',
 65: 'to',
 66: 'be',
 67: 'really',
 68: 'popular',
 69: 'when',
 70: 'started',
 71: 'high',
 72: 'school',
 73: 'then',
 74: 'it',
 75: 'was',
 76: 'ju

### Adding the End Of String token to the end of every answer

In [27]:
for i in range(len(clean_answers)):
    clean_answers[i] += '<EOS>'
    
clean_answers

['well i thought we would start with pronunciation if that is okay with you<EOS>',
 'not the hacking and gagging and spitting part  please<EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night<EOS>",
 'forget it<EOS>',
 'cameron<EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i cannot date until she does<EOS>',
 'seems like she could get a date easy enough<EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something<EOS>',
 'that is a shame<EOS>',
 'let me see what i can do<EOS>',
 'right  see  you are ready for the quiz<EOS>',
 "i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone<EOS>",
 "that is because it's such a nice one<EOS>",
 'forget french<EOS>',
 "well there's some

#### Translating all the questions and the answers into integers
#### and Replacing all the words that were filtered out by \<OUT>

`That is remove all the non frequent word(5% word) from questions list and answers list`

In [28]:
questions_into_int = []
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_into_int.append(ints)
answers_into_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_into_int.append(ints)

In [29]:
print(questionswords2int)


{'can': 0, 'we': 1, 'make': 2, 'this': 3, 'quick': 4, 'and': 5, 'andrew': 6, 'are': 7, 'having': 8, 'an': 9, 'incredibly': 10, 'public': 11, 'break': 12, 'up': 13, 'on': 14, 'the': 15, 'again': 16, 'well': 17, 'i': 18, 'thought': 19, 'would': 20, 'start': 21, 'with': 22, 'if': 23, 'that': 24, 'is': 25, 'okay': 26, 'you': 27, 'not': 28, 'part': 29, 'please': 30, 'asking': 31, 'me': 32, 'out': 33, 'so': 34, 'cute': 35, 'what': 36, 'your': 37, 'name': 38, 'no': 39, "it's": 40, 'my': 41, 'fault': 42, "didn't": 43, 'have': 44, 'a': 45, 'proper': 46, 'cameron': 47, 'thing': 48, 'am': 49, 'at': 50, 'mercy': 51, 'of': 52, 'particularly': 53, 'breed': 54, 'loser': 55, 'sister': 56, 'cannot': 57, 'date': 58, 'until': 59, 'she': 60, 'does': 61, 'why': 62, 'mystery': 63, 'used': 64, 'to': 65, 'be': 66, 'really': 67, 'popular': 68, 'when': 69, 'started': 70, 'high': 71, 'school': 72, 'then': 73, 'it': 74, 'was': 75, 'just': 76, 'like': 77, 'got': 78, 'sick': 79, 'or': 80, 'something': 81, 'gosh': 8

In [30]:
questions_into_int

[[0,
  1,
  2,
  3,
  4,
  8824,
  8824,
  5,
  6,
  8824,
  7,
  8,
  9,
  10,
  8824,
  11,
  12,
  13,
  14,
  15,
  8824,
  16],
 [17, 18, 19, 1, 20, 21, 22, 8824, 23, 24, 25, 26, 22, 27],
 [28, 15, 8824, 5, 8824, 5, 8824, 29, 30],
 [27, 7, 31, 32, 33, 24, 25, 34, 35, 36, 25, 37, 38, 16],
 [39, 39, 40, 41, 42, 1, 43, 44, 45, 46, 8824],
 [47],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  61],
 [62],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  81],
 [82, 23, 83, 1, 84, 85, 86, 45, 87],
 [8824, 88, 8824, 3, 25, 41, 89],
 [90, 91, 27, 7, 92, 93, 15, 8824],
 [18,
  94,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112,
  113,
 

In [31]:
print("indxx of removed words\n",ints)

indxx of removed words
 [34, 539, 83, 380, 6075, 135, 1, 44, 113, 1908, 52, 45, 328, 8824, 6069, 369, 462, 123, 8823]


In [32]:
print(answerswords2int)


{'can': 0, 'we': 1, 'make': 2, 'this': 3, 'quick': 4, 'and': 5, 'andrew': 6, 'are': 7, 'having': 8, 'an': 9, 'incredibly': 10, 'public': 11, 'break': 12, 'up': 13, 'on': 14, 'the': 15, 'again': 16, 'well': 17, 'i': 18, 'thought': 19, 'would': 20, 'start': 21, 'with': 22, 'if': 23, 'that': 24, 'is': 25, 'okay': 26, 'you': 27, 'not': 28, 'part': 29, 'please': 30, 'asking': 31, 'me': 32, 'out': 33, 'so': 34, 'cute': 35, 'what': 36, 'your': 37, 'name': 38, 'no': 39, "it's": 40, 'my': 41, 'fault': 42, "didn't": 43, 'have': 44, 'a': 45, 'proper': 46, 'cameron': 47, 'thing': 48, 'am': 49, 'at': 50, 'mercy': 51, 'of': 52, 'particularly': 53, 'breed': 54, 'loser': 55, 'sister': 56, 'cannot': 57, 'date': 58, 'until': 59, 'she': 60, 'does': 61, 'why': 62, 'mystery': 63, 'used': 64, 'to': 65, 'be': 66, 'really': 67, 'popular': 68, 'when': 69, 'started': 70, 'high': 71, 'school': 72, 'then': 73, 'it': 74, 'was': 75, 'just': 76, 'like': 77, 'got': 78, 'sick': 79, 'or': 80, 'something': 81, 'gosh': 8

In [33]:
answers_into_int

[[17, 18, 19, 1, 20, 21, 22, 8824, 23, 24, 25, 26, 22, 8824],
 [28, 15, 8824, 5, 8824, 5, 8824, 29, 8824],
 [26, 73, 97, 1533, 1, 860, 33, 482, 387, 8824, 210, 8824],
 [245, 8824],
 [8824],
 [15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  8824],
 [399, 77, 60, 84, 129, 45, 58, 865, 8824],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  8824],
 [24, 25, 45, 8824],
 [287, 32, 91, 36, 18, 0, 8824],
 [90, 91, 27, 7, 92, 93, 15, 8824],
 [18,
  94,
  95,
  65,
  96,
  97,
  65,
  98,
  24,
  99,
  18,
  95,
  65,
  96,
  100,
  101,
  77,
  102,
  15,
  103,
  104,
  7,
  97,
  105,
  61,
  106,
  107,
  108,
  77,
  109,
  18,
  44,
  110,
  111,
  41,
  112,
  113,
  65,
  114,
  33,
  41,
  89,
  65,
  8824],
 [24, 25, 116, 40, 117, 45, 118, 8824],
 [245, 8824],
 [17, 316, 115, 18, 194, 239, 66, 882

### Sorting questions and answers by the length of questions

In [34]:
sorted_clean_questions = []
sorted_clean_answers = []

for length in range(1,25+1):
    for i in enumerate(questions_into_int): # give index of the question and question that is converted into number
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_into_int[i[0]])
            sorted_clean_answers.append(answers_into_int[i[0]])

In [35]:
sorted_clean_questions

[[47],
 [62],
 [123],
 [147],
 [135],
 [39],
 [175],
 [39],
 [182],
 [183],
 [221],
 [36],
 [62],
 [135],
 [62],
 [110],
 [375],
 [132],
 [297],
 [211],
 [36],
 [222],
 [182],
 [26],
 [62],
 [182],
 [8824],
 [455],
 [250],
 [182],
 [193],
 [8824],
 [8824],
 [669],
 [97],
 [8824],
 [8824],
 [36],
 [39],
 [36],
 [8824],
 [147],
 [90],
 [36],
 [771],
 [8824],
 [637],
 [39],
 [39],
 [939],
 [8824],
 [1121],
 [39],
 [231],
 [69],
 [211],
 [39],
 [142],
 [1267],
 [211],
 [1113],
 [1113],
 [1113],
 [1113],
 [211],
 [340],
 [149],
 [26],
 [92],
 [669],
 [36],
 [1140],
 [1262],
 [8824],
 [8824],
 [1511],
 [211],
 [1552],
 [36],
 [36],
 [211],
 [1601],
 [1601],
 [1601],
 [1601],
 [1601],
 [1601],
 [26],
 [669],
 [67],
 [669],
 [231],
 [97],
 [1630],
 [1601],
 [1601],
 [1601],
 [8824],
 [1601],
 [1601],
 [67],
 [1671],
 [674],
 [1791],
 [211],
 [38],
 [211],
 [1224],
 [211],
 [1350],
 [211],
 [17],
 [123],
 [36],
 [62],
 [1840],
 [1773],
 [211],
 [1848],
 [211],
 [211],
 [1823],
 [222],
 [1148],


In [36]:
sorted_clean_answers

[[15,
  48,
  25,
  47,
  18,
  49,
  50,
  15,
  51,
  52,
  45,
  53,
  8824,
  54,
  52,
  55,
  41,
  56,
  18,
  57,
  58,
  59,
  60,
  8824],
 [8824,
  63,
  60,
  64,
  65,
  66,
  67,
  68,
  69,
  60,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  60,
  78,
  79,
  52,
  74,
  80,
  8824],
 [8824],
 [1529, 77, 101, 1550, 33, 149, 8824],
 [27, 153, 227, 3, 8824],
 [26, 27, 7, 160, 253, 65, 1280, 97, 65, 8824],
 [1387, 8824],
 [27, 239, 133, 194, 226, 8824],
 [8824],
 [20, 27, 124, 612, 32, 45, 1512, 8824],
 [41, 2582, 157, 18, 44, 78, 45, 103, 1112, 50, 963, 15, 8824, 144, 219, 8824],
 [111, 8824, 93, 45, 8824],
 [180, 75, 77, 45, 272, 8824],
 [279,
  24,
  18,
  280,
  18,
  20,
  110,
  128,
  281,
  76,
  116,
  278,
  267,
  75,
  264,
  74,
  5,
  18,
  282,
  283,
  284,
  93,
  8824,
  152,
  5,
  41,
  8824,
  8824,
  285,
  8823],
 [116, 60, 244, 4625, 383, 8824],
 [8824, 358, 119, 359, 360, 361, 15, 362, 119, 363, 364, 5, 8824],
 [129, 8824],
 [73, 134, 129, 8824]

# ` Building Seq to Seq model `
### Creating placeholders for the inputs and the targets

In [37]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None,None], name = 'input')
    targets = tf.placeholder(tf.int32, [None,None], name = 'target') # [None,None] represents targets variable is of two dimension
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32,name = 'keep_prob')
    
    return inputs,targets,lr,keep_prob

### Preprocessing the target

In [38]:
def preprocess_targets(targets,word2int,batch_size):
    left_side = tf.fill([batch_size,1],word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size,-1],[1,1]) #Extract everything except the last column
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

### Creating the Encoder RNN

In [39]:
def encoder_rnn(rnn_inputs,rnn_size,num_layers,keep_prob,seq_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm,input_keep_prob=keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output,encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                  cell_bw = encoder_cell,
                                                                  sequence_length = sequence_length,
                                                                  inputs = rnn_inputs,
                                                                  dtype = tf.float32)
    
    return encoder_state

### Decoding the training set


In [40]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

### Decoding the validation set

In [41]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions

### Creating the Decoder RNN

In [43]:
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions


### Building the seq2seq model

In [44]:
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions

In [ ]:
! git add Chatbot.ipynb 
! git commit -m "Building Seq to seq model"
! git push origin main